In [1]:
import datetime
import xarray  as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [29]:
def open_multiple_nc(nc_files):
    ds = xr.open_mfdataset(nc_files, concat_dim='nRec', combine='nested', parallel=True)

    #ds = ds.sel(nRec=slice(16500, 17000))

    return ds

dir = '/home/sachin/Documents/NIPR/Research/Data/AMPERE/raw/SH/2015/*.nc'
open_multi_amp = open_multiple_nc(dir)
open_multi_amp


<xarray.Dataset> Size: 65GB
Dimensions:       (nRec: 261234, nObs: 1200, vComp: 3)
Dimensions without coordinates: nRec, nObs, vComp
Data variables: (12/34)
    npnt          (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    year          (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    doy           (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    time          (nRec) float32 1MB dask.array<chunksize=(720,), meta=np.ndarray>
    avgint        (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    kmax          (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    ...            ...
    del_db_geo    (nRec, nObs, vComp) float64 8GB dask.array<chunksize=(720, 1200, 3), meta=np.ndarray>
    del_jPar      (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Th  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Th  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Ph  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Ph  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
Attributes:
    title:        Ampere data for 2015-01-01 00:00 - 2015-01-02 00:00.
    description:  AMPERE GRD data (fitted magnetic pertubation data).
    created:      2022-09-11T23:14:16Z

In [30]:
def create_datetime(ds):

    ds = ds.copy()
    fractional_hours = ds['time'].values
    year = ds['year'].values
    doy = ds['doy'].values

    df = pd.DataFrame({'fractional_hours': fractional_hours})
    df['hours'] = df['fractional_hours'].apply(lambda x: int(x))
    df['minutes'] = df['fractional_hours'].apply(lambda x: int((x * 60) % 60))
    #round up minutes to nearest 2
    df['minutes'] = df['minutes'].apply(lambda x: x + 1 if x % 2 != 0 else x)

    df['time'] = df.apply(lambda row: f"{int(row['hours']):02}:{int(row['minutes']):02}", axis=1)
    df['doy'] = doy
    df['doy'] = df['doy'].astype(int)
    df['year'] = year
    df['year'] = df['year'].astype(int)

    def doy_year_to_date(year, doy):
        return datetime.datetime(year, 1, 1) + datetime.timedelta(int(doy) - 1)

    df['date'] = df.apply(lambda row: doy_year_to_date(row['year'], row['doy']), axis=1)
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    df['dt'] = pd.to_datetime(df['date'] + ' ' + df['time'])

    #add dt to ds
    ds['dt'] = df['dt']
    ds = ds.sortby('dt')

    return ds

open_amp = create_datetime(open_multi_amp)
open_amp

<xarray.Dataset> Size: 65GB
Dimensions:       (nRec: 261234, nObs: 1200, vComp: 3, dim_0: 261234)
Coordinates:
  * dim_0         (dim_0) int64 2MB 0 1 2 3 4 ... 261230 261231 261232 261233
Dimensions without coordinates: nRec, nObs, vComp
Data variables: (12/35)
    npnt          (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    year          (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    doy           (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    time          (nRec) float32 1MB dask.array<chunksize=(720,), meta=np.ndarray>
    avgint        (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    kmax          (nRec) int16 522kB dask.array<chunksize=(720,), meta=np.ndarray>
    ...            ...
    del_jPar      (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Th  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Th  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Th_Ph  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    del_db_Ph_Ph  (nRec, nObs) float64 3GB dask.array<chunksize=(720, 1200), meta=np.ndarray>
    dt            (dim_0) datetime64[ns] 2MB 2015-01-01 ... 2015-12-31T23:58:00
Attributes:
    title:        Ampere data for 2015-01-01 00:00 - 2015-01-02 00:00.
    description:  AMPERE GRD data (fitted magnetic pertubation data).
    created:      2022-09-11T23:14:16Z

In [31]:
def create_new_ds(ds):

    ds = ds.copy()
    jPar = ds['jPar'].values
    dt = ds['dt'].values
    mlat = np.linspace(40, 90, 50)
    mlt = np.linspace(0, 24, 24)

    ds_new = xr.Dataset({'jPar': (['dt','mlat','mlt'], jPar.reshape(-1, 50, 24))},
                        coords={'dt': dt,'mlat': mlat,'mlt': mlt})
    
    #average over x min
    ds_new = ds_new.resample(dt='10min').mean()
    ds_new = ds_new.interpolate_na(dim='dt')
    
    #ds_new = ds_new.sel(dt=slice('2010-08-01 00:00:00', '2010-08-01 23:30:00'))

    return ds_new

ds = create_new_ds(open_amp)
ds

<xarray.Dataset> Size: 505MB
Dimensions:  (mlat: 50, mlt: 24, dt: 52560)
Coordinates:
  * mlat     (mlat) float64 400B 40.0 41.02 42.04 43.06 ... 87.96 88.98 90.0
  * mlt      (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 20.87 21.91 22.96 24.0
  * dt       (dt) datetime64[ns] 420kB 2015-01-01 ... 2015-12-31T23:50:00
Data variables:
    jPar     (dt, mlat, mlt) float64 505MB 0.07791 0.1556 ... 0.05055 0.006394

In [32]:
def get_omni(df):
    df = df.copy()
    df['dt'] = df['Timestamp_1min_omni']
    df['dt'] = pd.to_datetime(df['dt'])
    df = df.set_index('dt')
    df = df.drop(columns=['Timestamp_1min_omni','Timestamp_1hr_omni'])

    ds = xr.Dataset.from_dataframe(df)
    ds = ds.sortby('dt')

    return ds

omni_ds = get_omni(pd.read_csv('/home/sachin/Documents/NIPR/Research/Data/OMNI/omni_hro_5min_2015.csv'))
omni_ds

<xarray.Dataset> Size: 7MB
Dimensions:         (dt: 103533)
Coordinates:
  * dt              (dt) datetime64[ns] 828kB 2015-01-01 ... 2015-12-31T23:55:00
Data variables:
    BX_GSE          (dt) float64 828kB 5.35 6.19 6.28 6.27 ... 5.14 4.99 4.8 4.8
    BY_GSE          (dt) float64 828kB -2.31 -1.17 -1.81 ... -0.96 -0.79 -0.66
    BZ_GSE          (dt) float64 828kB 1.85 2.39 2.56 ... -14.59 -14.62 -14.59
    flow_speed      (dt) int64 828kB 531 544 560 562 568 ... 461 457 454 456 455
    proton_density  (dt) float64 828kB 2.92 2.43 1.7 1.69 ... 2.6 2.45 2.9 2.64
    F10.7           (dt) int64 828kB 132 132 132 132 132 132 ... 93 93 93 93 93
    Kp              (dt) int64 828kB 23 23 23 23 23 23 23 ... 57 57 57 57 57 57
    doy             (dt) int64 828kB 1 1 1 1 1 1 1 ... 365 365 365 365 365 365

In [33]:
#merge omni and ampere data
def merge_omni_ampere(omni_ds, ampere_ds):
    ds = xr.merge([omni_ds, ampere_ds], join='inner')
    #ds = ds.sel(dt=slice('2010-08-01','2010-08-02'))
    #save ds_new to netcdf
    ds = ds.sel(mlat=slice(40, 90))
    ds = ds.sortby('dt')

    ds.to_netcdf('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/SH/ampere_omni_2015_10min.nc')
    #ds.to_netcdf('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/excluded/20130514.nc')
    
    return ds

merge_omni_ampere(omni_ds, ds)

<xarray.Dataset> Size: 501MB
Dimensions:         (dt: 51772, mlat: 50, mlt: 24)
Coordinates:
  * dt              (dt) datetime64[ns] 414kB 2015-01-01 ... 2015-12-31T23:50:00
  * mlat            (mlat) float64 400B 40.0 41.02 42.04 ... 87.96 88.98 90.0
  * mlt             (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 21.91 22.96 24.0
Data variables:
    BX_GSE          (dt) float64 414kB 5.35 6.28 5.82 4.6 ... 5.3 5.81 5.14 4.8
    BY_GSE          (dt) float64 414kB -2.31 -1.81 -2.22 ... -2.36 -1.17 -0.79
    BZ_GSE          (dt) float64 414kB 1.85 2.56 1.66 ... -14.27 -14.57 -14.62
    flow_speed      (dt) int64 414kB 531 560 568 553 552 ... 459 460 461 457 456
    proton_density  (dt) float64 414kB 2.92 1.7 1.95 3.08 ... 3.73 2.79 2.6 2.9
    F10.7           (dt) int64 414kB 132 132 132 132 132 132 ... 93 93 93 93 93
    Kp              (dt) int64 414kB 23 23 23 23 23 23 23 ... 57 57 57 57 57 57
    doy             (dt) int64 414kB 1 1 1 1 1 1 1 ... 365 365 365 365 365 365
    jPar            (dt, mlat, mlt) float64 497MB 0.07791 0.1556 ... 0.006394

In [34]:
def nan_count(ds, var):
    #count number of NaNs in a data variable
    con_data = ds[var].values
    nan_count = np.count_nonzero(np.isnan(con_data))
    nan_ratio = nan_count / con_data.size
    not_nan = con_data.size - nan_count
    return not_nan, nan_count, np.round(nan_ratio, 2)

nan_count(merge_omni_ampere(omni_ds, ds), 'jPar')

(62126400, 0, 0.0)

In [38]:
def merge_ds():
    ds_2010 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/SH/ampere_omni_2010_10min.nc')
    ds_2011 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/SH/ampere_omni_2011_10min.nc')
    ds_2013 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/SH/ampere_omni_2013_10min.nc')
    ds_2015 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/SH/ampere_omni_2015_10min.nc')
    #ds_2017 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2017_5min.nc')
    #ds_2019 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2019_5min.nc')
    #ds_2021 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2021_5min.nc')

    ds = xr.concat([ds_2010, ds_2011, ds_2013, ds_2015], dim='dt')    
    #ds = ds.sel(mlat=slice(50, 90))
    ds = ds.sortby('dt')
    ds = ds.interpolate_na(dim='dt')

    print(nan_count(ds, 'jPar'))

    ds.to_netcdf('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/SH/ampere_omni_2010_2011_2013_2015_10min.nc')

    #return ds
    return ds

merge_ds()

(242953200, 0, 0.0)


<xarray.Dataset> Size: 2GB
Dimensions:         (dt: 202461, mlat: 50, mlt: 24)
Coordinates:
  * dt              (dt) datetime64[ns] 2MB 2010-01-01 ... 2015-12-31T23:50:00
  * mlat            (mlat) float64 400B 40.0 41.02 42.04 ... 87.96 88.98 90.0
  * mlt             (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 21.91 22.96 24.0
Data variables:
    jPar            (dt, mlat, mlt) float64 2GB -0.05808 -0.1187 ... 0.006394
    BX_GSE          (dt) float64 2MB 0.08 0.16 -0.1 -0.11 ... 5.3 5.81 5.14 4.8
    BY_GSE          (dt) float64 2MB 2.99 3.11 2.92 2.59 ... -2.36 -1.17 -0.79
    BZ_GSE          (dt) float64 2MB -0.37 0.25 0.69 ... -14.27 -14.57 -14.62
    flow_speed      (dt) int32 810kB 283 284 282 283 283 ... 459 460 461 457 456
    proton_density  (dt) float64 2MB 3.99 3.85 3.81 3.65 ... 3.73 2.79 2.6 2.9
    F10.7           (dt) int32 810kB 72 72 72 72 72 72 72 ... 93 93 93 93 93 93
    Kp              (dt) int32 810kB 0 0 0 0 0 0 0 0 ... 57 57 57 57 57 57 57 57
    doy             (dt) int32 810kB 1 1 1 1 1 1 1 ... 365 365 365 365 365 365

In [48]:
def merge_ds():
    ds_2009 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2009_5min.nc')
    ds_2010 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2010_5min.nc')
    ds_2011 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2011_5min.nc')
    ds_2012 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2012_5min.nc')
    ds_2013 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2013_5min.nc')
    ds_2014 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2014_5min.nc')
    ds_2015 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2015_5min.nc')
    ds_2016 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2016_5min.nc')
    ds_2017 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2017_5min.nc')
    ds_2018 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2018_5min.nc')
    ds_2019 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2019_5min.nc')
    ds_2020 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2020_5min.nc')
    ds_2021 = xr.open_dataset('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2021_5min.nc')

    ds = xr.concat([ds_2009,  ds_2010, ds_2011, ds_2012, ds_2013, ds_2014, ds_2015, ds_2016, ds_2017, ds_2018, ds_2020, ds_2021], dim='dt')
    
    #ds = ds.sel(mlat=slice(50, 90))
    ds = ds.sortby('dt')
    ds = ds.interpolate_na(dim='dt')

    print(nan_count(ds, 'jPar'))

    ds.to_netcdf('/home/sachin/Documents/NIPR/Research/Data/AMPERE/processed/ampere_omni_2009_2021_5min.nc')

    #return ds
    return ds

#merge_ds()

(1269676800, 0, 0.0)


<xarray.Dataset> Size: 10GB
Dimensions:         (dt: 1058064, mlat: 50, mlt: 24)
Coordinates:
  * dt              (dt) datetime64[ns] 8MB 2009-12-01 ... 2021-12-31T23:50:00
  * mlat            (mlat) float64 400B 40.0 41.02 42.04 ... 87.96 88.98 90.0
  * mlt             (mlt) float64 192B 0.0 1.043 2.087 3.13 ... 21.91 22.96 24.0
Data variables:
    jPar            (dt, mlat, mlt) float64 10GB 0.01381 0.0407 ... -0.03643
    BX_GSE          (dt) float64 8MB -2.01 -1.72 -1.4 ... -4.53 -4.46 -3.68
    BY_GSE          (dt) float64 8MB 0.7 1.24 1.63 1.64 ... 3.22 3.28 2.89 2.19
    BZ_GSE          (dt) float64 8MB -0.13 0.09 0.34 -0.36 ... -4.17 -4.76 -5.41
    flow_speed      (dt) int32 4MB 275 276 277 276 276 ... 398 395 395 397 399
    proton_density  (dt) float64 8MB 6.06 5.88 6.03 6.0 ... 7.14 6.91 7.32 7.72
    F10.7           (dt) int32 4MB 70 70 70 70 70 70 70 ... 98 98 98 98 98 98 98
    Kp              (dt) int32 4MB 0 0 0 0 0 0 0 0 0 ... 10 10 10 10 10 10 10 10
    doy             (dt) int32 4MB 335 335 335 335 335 ... 365 365 365 365 365

In [ ]:
fac = ds['jPar'].mean(dim='dt').values

def dt_to_str(dt):
    dt_1 = dt[0]
    dt_2 = dt[1] 
    time_1 = pd.to_datetime(dt_1)
    time_2 = pd.to_datetime(dt_2)
    str_time_1 = time_1.strftime('%Y-%m-%d %H:%M')
    str_time_2 = time_2.strftime('%Y-%m-%d %H:%M')

    return str_time_1, str_time_2

dt = open_amp['dt'].values
start_time, end_time = dt_to_str(dt)

fac = fac.reshape(24, 50).T # reshape and transpose
fac = np.flipud(fac) # flip the array upside down
theta = np.linspace(0, 360, 24) - 90 # rotate by 90 degrees
theta = np.radians(theta) # convert to radians
r = 90 - np.linspace(40, 90, 50) #convert to colat

cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", ["blue","blue", "white", "white","red","red"])
cmap = 'bwr'

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(6, 5))

c = ax.contourf(theta, r, fac, cmap=cmap, levels=np.linspace(-0.5,.5 , 100))
ax.set_ylim([0,40])
ax.set_yticks([0, 10, 20, 30,40])
ax.set_yticklabels(["90°", "80°", "70°", "60°","50° MLAT"])
ax.set_xlim([-np.pi, np.pi])
ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
ax.set_xticklabels(["21", "0 MLT \nMidnight", "3", "6 \n  Dawn", "9", "12 MLT \nMidday", "15", "18 \nDusk"])
ax.grid(True, linestyle='-.', alpha=0.7)
ax.set_title(f"{start_time} - {end_time}", pad=10, fontsize=11.5)


plt.colorbar(c, ax=ax, label='J$_\parallel$ (FAC) [µA/m$^2$]', shrink=0.3, pad = 0.12, 
             ticks=[-0.5, 0,  0.5], 
             orientation='horizontal')

plt.tight_layout()
#plt.savefig('ampere_20100807.png', dpi=400)